In [1]:
import pandas as pd
from tqdm.auto import tqdm
import os
import random
import numpy as np

def get_errors_and_duplicate_samle(data_url = "double_checked_data.csv"):
    data = pd.read_csv(data_url)
    def cut_video(implementer,title,idx,order_id,label_id,start_time,end_time,raw_video_name):
        start_time = float(start_time)
        end_time = float(end_time)
        implementer_id = implementer.split("_")[0]
        title_ = None
        if 'right' in title:
            title_ = 'right'
        elif 'left' in title:
            title_ = 'left'
        elif 'center' in title:
            title_ = 'center'
        assert title_ is not None

        name = f"{raw_video_name}_signer{implementer_id}_{title_}_ord{int(order_id)}_{int(label_id)-1}.mp4"

        return [name,label_id-1]


    labels = []
    errors = []
    for idx,(group_id, group_data) in tqdm(enumerate(data.groupby("id"))):
        for id,(i,row) in enumerate(group_data.iterrows()):
            try:
                lb = cut_video(
                                row['implementer'], row['title'], id+1, int(row['order_action']), int(row['label_id']),
                                row['start_time'], row['end_time'],row['title']
                            )
                lb.extend([row['id-2']])
                labels.append(lb)
            except Exception as e:
                errors.append([group_id,row['start_time'], row['end_time'],e])
    errors = pd.DataFrame(errors,columns=['video_id','start',"end","error"])
    labels = pd.DataFrame(labels,columns=['name','label',"video_lb_id"])

    # remove duplicated video
    duplicate_labels = labels[labels.duplicated(subset=['name'])]
    duplicate_labels = duplicate_labels.merge(data[['id-2', 'id']], left_on='video_lb_id', right_on='id-2', how='left')
    # Xóa cột 'col_name' từ DataFrame
    duplicate_labels.drop(columns=['id-2'], inplace=True)
    duplicate_labels['label'] += 1
    return errors,duplicate_labels

In [2]:
errors,duplicate_labels = get_errors_and_duplicate_samle(data_url = "data.csv")

0it [00:00, ?it/s]

In [9]:
duplicate_labels.to_csv("duplicate_videos.csv",index=False)

In [11]:
errors.to_csv("errors.csv",index=False)

In [6]:
duplicate_labels

,name,label,video_lb_id,id
0,16_Tuan-Kiet_400-700_10-11-12_0112___center_de...,654,763639,16
1,08_Thay-Son_400-700_1-2-3_0112___center_device...,421,758534,24
2,08_Thay-Son_400-700_1-2-3_0112___center_device...,467,758581,24
3,04_Co-Hang_700-1000_13-14-15_0122___right_devi...,715,506518,40
4,04_Co-Hang_700-1000_13-14-15_0122___right_devi...,862,506665,40
...,...,...,...,...
1150,16_Tuan-Kiet_700-1000_4-5-6_0119___center_devi...,998,790475,431
1151,16_Tuan-Kiet_700-1000_4-5-6_0119___center_devi...,999,790192,431
1152,16_Tuan-Kiet_700-1000_4-5-6_0119___center_devi...,1000,790477,431
1153,16_Tuan-Kiet_700-1000_4-5-6_0119___center_devi...,1001,790194,431


In [4]:
errors

,video_id,start,end,error
